# 01. 다양한 LLM 활용

In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [13]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


## OpenAI

OpenAI는 **채팅 전용 Large Language Model (LLM)** 을 제공합니다. 이 모델을 생성할 때 다양한 옵션을 지정할 수 있으며, 이러한 옵션들은 모델의 동작 방식에 영향을 미칩니다

옵션 상세 설명
1. `temperature`

**샘플링 온도**를 설정하는 옵션입니다. 값은 0과 2 사이에서 선택할 수 있습니다. 높은 값(예: 0.8)은 출력을 더 무작위하게 만들고, 낮은 값(예: 0.2)은 출력을 더 집중되고 결정론적으로 만듭니다.

2. `max_tokens`

채팅 완성에서 생성할 **토큰의 최대 개수**를 지정합니다. 이 옵션은 모델이 한 번에 생성할 수 있는 텍스트의 길이를 제어합니다.

3. `model_name`
적용 가능한 모델을 선택하는 옵션입니다. 더 자세한 정보는 OpenAI 모델 문서에서 확인할 수 있습니다.

4. 모델 스펙

링크: https://platform.openai.com/docs/models/gpt-4o

In [14]:
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response

# ChatOpenAI 객체를 생성합니다.
gpt = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o",  # 모델명
)

# 스트리밍 출력을 위하여 invoke() 대신 stream()을 사용합니다.
answer = gpt.stream("사랑이 뭔가요?")

# 답변 출력
stream_response(answer)


ImportError: cannot import name 'LangSmithParams' from 'langchain_core.language_models.chat_models' (C:\Users\osca0\PycharmProjects\StudyLangChain\.venv\Lib\site-packages\langchain_core\language_models\chat_models.py)

In [16]:
from langchain_anthropic import ChatAnthropic

# ChatAnthropic 객체를 생성합니다.
anthropic = ChatAnthropic(model_name="claude-3-5-sonnet-20241022")

# 스트리밍 출력을 위하여 invoke() 대신 stream()을 사용합니다.
answer = anthropic.stream("사랑이 뭔가요?")

# 답변 출력
stream_response(answer)


ImportError: cannot import name 'LangSmithParams' from 'langchain_core.language_models.chat_models' (C:\Users\osca0\PycharmProjects\StudyLangChain\.venv\Lib\site-packages\langchain_core\language_models\chat_models.py)

# 02. 캐싱(Cache)

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 모델을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘")

# 체인을 생성합니다.
chain = prompt | llm


In [9]:
%%time
response = chain.invoke({"country": "한국"})
print(response.content)


한국은 동아시아에 위치한 고도 경제대국으로, 문화와 역사적으로도 다양한 매력을 지니고 있다. 대한민국과 북한으로 나뉘어 있으며, 수도는 서울에 위치해 있다. 산업화와 IT 기술 발전으로 세계에서 주요 경제 국가로 발전하였고, K-pop이나 K-drama 등의 문화 산업 역시 글로벌한 인기를 누리고 있다. 또한, 전통적인 문화와 성장한 대도시들 사이의 조화로움이 독특한 매력을 지니고 있으며, 맛있는 음식과 아름다운 자연환경도 많이 갖고 있다. 현대적인 발전과 전통적인 가치를 동시에 제공하는 한국은 매년 많은 관광객들의 방문을 유치하고 있다.
CPU times: total: 15.6 ms
Wall time: 4.32 s


## InMemoryCache

인메모리 캐시를 사용하여 동일 질문에 대한 답변을 저장하고, 캐시에 저장된 답변을 반환

In [10]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시를 사용합니다.
set_llm_cache(InMemoryCache())

# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)


한국은 동아시아에 위치한 반도 국가로, 수도는 서울이다.  대한민국과 북한으로 나뉘어져 있으며, 한국어를 공용어로 사용한다. 높은 경제성장률과 혁신적인 기술력으로 세계적으로 주목받는 나라이다. 또한 한류 열풍을 일으키며 한국 문화뿐만 아니라 음악, 드라마, 영화 등이 전 세계에서 인기를 끌고 있다. 한국은 고대부터 현대까지 다채로운 역사와 전통 문화를 지녔으며, 한반도의 아름다운 자연환경과 맛있는 음식으로도 유명하다. 현재는 선진화된 도시와 전통적인 문화가 공존하는 다양한 모습을 보여주고 있다.
CPU times: total: 219 ms
Wall time: 3.53 s


두번쨰로 실행할떄는 CPU Times가 0 ns 로 나온다

In [11]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)


한국은 동아시아에 위치한 반도 국가로, 수도는 서울이다.  대한민국과 북한으로 나뉘어져 있으며, 한국어를 공용어로 사용한다. 높은 경제성장률과 혁신적인 기술력으로 세계적으로 주목받는 나라이다. 또한 한류 열풍을 일으키며 한국 문화뿐만 아니라 음악, 드라마, 영화 등이 전 세계에서 인기를 끌고 있다. 한국은 고대부터 현대까지 다채로운 역사와 전통 문화를 지녔으며, 한반도의 아름다운 자연환경과 맛있는 음식으로도 유명하다. 현재는 선진화된 도시와 전통적인 문화가 공존하는 다양한 모습을 보여주고 있다.
CPU times: total: 0 ns
Wall time: 5.76 ms


## SQLite Cache

In [12]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# 캐시 디렉토리를 생성합니다.
if not os.path.exists("cache"):
    os.makedirs("cache")

# SQLiteCache를 사용합니다.
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))


In [15]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)


한국은 동아시아에 위치한 고도로 발달한 경제를 갖춘 선진국가이다. 주요 산업은 자동차, 전자제품, 선박 등이며 세계적으로 유명한 기업들이 많이 위치해 있다. 문화적으로는 한글과 한국 전통 의상, 음식 등이 유명하며 K-pop, K-drama 등 한류 열풍이 전 세계적으로 인기를 끌고 있다. 또한 이곳에서는 서로를 존경하고 예의 바른 태도를 중요시하는 문화가 뿌리깊게 자리잡고 있다. 정치적으로는 북한과 한반도 문제, 미국과의 관계 등 여러 이슈가 있지만 안정적으로 발전하며 세계에서 주목받는 나라 중 하나이다.
CPU times: total: 15.6 ms
Wall time: 11.5 ms


마찬가지로 재실행할 시 저장된 값이 그대로 출력된다.

In [12]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 반도 국가로, 수도는 서울이다.  대한민국과 북한으로 나뉘어져 있으며, 한국어를 공용어로 사용한다. 높은 경제성장률과 혁신적인 기술력으로 세계적으로 주목받는 나라이다. 또한 한류 열풍을 일으키며 한국 문화뿐만 아니라 음악, 드라마, 영화 등이 전 세계에서 인기를 끌고 있다. 한국은 고대부터 현대까지 다채로운 역사와 전통 문화를 지녔으며, 한반도의 아름다운 자연환경과 맛있는 음식으로도 유명하다. 현재는 선진화된 도시와 전통적인 문화가 공존하는 다양한 모습을 보여주고 있다.
CPU times: total: 15.6 ms
Wall time: 5.5 ms


# 03. 모델 직렬화(Serialization) - 저장 및 불러오기

직렬화(Serialization)
직렬화(Serialization) 란?

정의

: 모델을 저장 가능한 형식으로 변환하는 과정
목적

- 모델 재사용 (재훈련 없이)
- 모델 배포 및 공유 용이
- 계산 리소스 절약

장점

- 빠른 모델 로딩
- 버전 관리 가능
- 다양한 환경에서 사용 가능

- 모델 직렬화는 AI 개발 및 배포 과정에서 중요한 단계로, 효율적인 모델 관리와 재사용을 가능하게 합니다.

is_lc_serializable 클래스 메서드로 실행하여 LangChain 클래스가 직렬화 가능한지 확인할 수 있습니다

In [13]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# 프롬프트 템플릿을 사용하여 질문을 생성합니다.
prompt = PromptTemplate.from_template("{fruit}의 색상이 무엇입니까?")


In [14]:
# 직렬화가 가능한지 체크합니다.
print(f"ChatOpenAI: {ChatOpenAI.is_lc_serializable()}")

ChatOpenAI: True


In [15]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 직렬화가 가능한지 체크합니다.
print(f"ChatOpenAI: {llm.is_lc_serializable()}")

ChatOpenAI: True


In [23]:
# 체인을 생성합니다.
chain = prompt | llm

# 직렬화가 가능한지 체크합니다.
chain.is_lc_serializable()

True

## 체인(Chain) 직렬화(dumps, dumpd)

체인 직렬화는 직렬화 가능한 모든 객체를 딕셔너리 또는 JSON 문자열로 변환하는 과정을 의미합니다.

직렬화 방법
- 객체의 속성 및 데이터를 키-값 쌍으로 저장하여 딕셔너리 형태로 변환합니다.

- 이러한 직렬화 방식은 객체를 쉽게 저장하고 전송할 수 있게 하며, 다양한 환경에서 객체를 재구성할 수 있도록 합니다.

참고

`dumps`: 객체를 JSON 문자열로 직렬화

`dumpd`: 객체를 딕셔너리로 직렬화

In [16]:
from langchain_core.load import dumpd, dumps

dumpd_chain = dumpd(chain)
dumpd_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['country'],
    'template': '{country} 에 대해서 200자 내외로 요약해줘',
    'template_format': 'f-string'},
   'name': 'PromptTemplate'},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
   'kwargs': {'model_name': 'gpt-3.5-turbo',
    'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']},
    'stream_usage': True,
    'output_version': 'v0'},
   'name': 'ChatOpenAI'}},
 'name': 'RunnableSequence'}

In [25]:
# 직렬화된 체인의 타입을 확인합니다.
type(dumpd_chain)

dict

In [26]:
# dumps 함수를 사용하여 직렬화된 체인을 확인합니다.
dumps_chain = dumps(chain)
dumps_chain

'{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwargs": {"first": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "prompt", "PromptTemplate"], "kwargs": {"input_variables": ["fruit"], "template": "{fruit}\\uc758 \\uc0c9\\uc0c1\\uc774 \\ubb34\\uc5c7\\uc785\\ub2c8\\uae4c?", "template_format": "f-string"}, "name": "PromptTemplate"}, "last": {"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"model_name": "gpt-3.5-turbo", "temperature": 0.0, "openai_api_key": {"lc": 1, "type": "secret", "id": ["OPENAI_API_KEY"]}, "output_version": "v0"}, "name": "ChatOpenAI"}}, "name": "RunnableSequence"}'

## Pickle 파일을 활용한 직렬화

In [27]:
import pickle

# fuit_chain.pkl 파일로 직렬화된 체인을 저장합니다.
with open("fruit_chain.pkl", "wb") as f:
    pickle.dump(dumpd_chain, f)


JSON을 활용하는 방법 또한 앞서 본 것처럼 가능하다

In [28]:
import json

with open("fruit_chain.json", "w") as fp:
    json.dump(dumpd_chain, fp)

## load: 저장한 모델 불러오기

In [29]:
import pickle

# pickle 파일을 로드합니다.
with open("fruit_chain.pkl", "rb") as f:
    loaded_chain = pickle.load(f)


In [30]:
from langchain_core.load import load

# 체인을 로드합니다.
chain_from_file = load(loaded_chain)

# 체인을 실행합니다.
print(chain_from_file.invoke({"fruit": "사과"}))


C:\Users\osca0\AppData\Local\Temp\ipykernel_20460\1490994347.py:4: LangChainBetaWarning: The function `load` is in beta. It is actively being worked on, so the API may change.
  chain_from_file = load(loaded_chain)


content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 24, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cc7oc0Y37967OIUs6na9uX3WDB0DZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--c515b5d6-83b5-4b0c-9b3f-bf9a0487cce2-0' usage_metadata={'input_tokens': 24, 'output_tokens': 51, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [32]:
from langchain_core.load import load, loads

load_chain = load(
    loaded_chain, secrets_map={"OPENAI_API_KEY": os.environ["OPENAI_API_KEY"]}
)

# 불러온 체인이 정상 동작하는지 확인합니다.
load_chain.invoke({"fruit": "사과"})


AIMessage(content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 24, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cc7oc0Y37967OIUs6na9uX3WDB0DZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c515b5d6-83b5-4b0c-9b3f-bf9a0487cce2-0', usage_metadata={'input_tokens': 24, 'output_tokens': 51, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}, 'total_cost': 0})

In [33]:
with open("fruit_chain.json", "r") as fp:
    loaded_from_json_chain = json.load(fp)
    loads_chain = load(loaded_from_json_chain)


In [34]:
# 불러온 체인이 정상 동작하는지 확인합니다.
loads_chain.invoke({"fruit": "사과"})

AIMessage(content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 24, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cc7oc0Y37967OIUs6na9uX3WDB0DZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c515b5d6-83b5-4b0c-9b3f-bf9a0487cce2-0', usage_metadata={'input_tokens': 24, 'output_tokens': 51, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}, 'total_cost': 0})

# 04. 토큰 사용량 확인

특정 호출에 대한 토큰 사용량을 추적하는 방법에 대해 설명합니다.

이 기능은 현재 OpenAI API 에만 구현되어 있습니다.

먼저 단일 Chat 모델 호출에 대한 토큰 사용량을 추적하는 매우 간단한 예를 살펴보겠습니다.

In [35]:
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

# 모델을 불러옵니다.
llm = ChatOpenAI(model_name="gpt-4o")

with get_openai_callback() 구문안에서 실행되는 모든 토큰 사용량/요금이 추적됩니다.

In [36]:
# callback을 사용하여 추적합니다.
with get_openai_callback() as cb:
    result = llm.invoke("대한민국의 수도는 어디야?")
    print(cb)


Tokens Used: 23
	Prompt Tokens: 15
		Prompt Tokens Cached: 0
	Completion Tokens: 8
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.00011750000000000001


In [37]:
# callback을 사용하여 추적합니다.
with get_openai_callback() as cb:
    result = llm.invoke("대한민국의 수도는 어디야?")
    result = llm.invoke("대한민국의 수도는 어디야?")
    print(f"총 사용된 토큰수: \t\t{cb.total_tokens}")
    print(f"프롬프트에 사용된 토큰수: \t{cb.prompt_tokens}")
    print(f"답변에 사용된 토큰수: \t{cb.completion_tokens}")
    print(f"호출에 청구된 금액(USD): \t${cb.total_cost}")

총 사용된 토큰수: 		46
프롬프트에 사용된 토큰수: 	30
답변에 사용된 토큰수: 	16
호출에 청구된 금액(USD): 	$0.00023500000000000002


# 05. 구글 생성 AI(Google Generative AI)

## Google AI chat models (gemini-pro)
- 패키지 버전 문제로 직접 실행 불가

Google AI의 gemini와 gemini-vision 모델뿐만 아니라 다른 생성 모델에 접근하려면 langchain-google-genai 통합 패키지의 ChatGoogleGenerativeAI 클래스를 사용하면 됩니다.

In [21]:
!pip install -qU langchain-google-genai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.3.2 requires google-ai-generativelanguage==0.4.0, but you have google-ai-generativelanguage 0.9.0 which is incompatible.
langchain 0.1.19 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 1.0.5 which is incompatible.
langchain 0.1.19 requires langsmith<0.2.0,>=0.1.17, but you have langsmith 0.4.43 which is incompatible.
langchain-anthropic 0.3.10 requires langchain-core<1.0.0,>=0.3.45, but you have langchain-core 1.0.5 which is incompatible.
langchain-azure-ai 0.1.2 requires langchain-core<0.4.0,>=0.3.0, but you have langchain-core 1.0.5 which is incompatible.
langchain-chroma 0.2.2 requires langchain-core!=0.3.0,!=0.3.1,!=0.3.10,!=0.3.11,!=0.3.12,!=0.3.13,!=0.3.14,!=0.3.2,!=0.3.3,!=0.3.4,!=0.3.5,!=0.3.6,!=0.3.7,!=0.3.8,!=0.3.9,<0.4.0,>=0.2.43, but you have langchain-core 1.

### 패키지 호환성 문제 해결

In [19]:
# 현재 버전을 삭제하고
!pip uninstall -y langchain-core langchain-google-genai langchain langchain-community langsmith
# 다운그레이드된 버전을 설치한다/
!pip install "langchain==0.1.19" "langchain-core==0.1.52" "langchain-google-genai==0.0.9" "google-generativeai==0.3.2" "google-ai-generativelanguage==0.4.0"

^C
  Using cached langchain-0.1.19-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_google_genai-0.0.9-py3-none-any.whl.metadata (2.9 kB)
  Using cached google_ai_generativelanguage-0.4.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_text_splitters-0.0.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
Using cached langchain-0.1.19-py3-none-any.whl (1.0 MB)
Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
Using cached langchain_google_genai-0.0.9-py3-none-any.whl (17 kB)
Using cached google_ai_generativelanguage-0.4.0-py3-none-any.whl (598 kB)
Using cached langchain_community-0.0.38-py3-none-any.whl (2.0 MB)
Using cached langchain_text_splitters-0.0.2-py3-none-any.whl (23 kB)
Using cached

ERROR: Could not install packages due to an OSError: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'C:\\Users\\osca0\\PycharmProjects\\StudyLangChain\\.venv\\Lib\\site-packages\\langsmith\\wrappers\\_openai.py'
Check the permissions.



In [20]:
from dotenv import load_dotenv

load_dotenv()


True

langchain_google_genai 패키지에서 ChatGoogleGenerativeAI 클래스를 가져옵니다.


ChatGoogleGenerativeAI 클래스는 Google의 Generative AI 모델을 사용하여 대화형 AI 시스템을 구현하는 데 사용됩니다.

이 클래스를 통해 사용자는 Google의 대화형 AI 모델과 상호 작용할 수 있습니다.

모델과의 대화는 채팅 형식으로 이루어지며, 사용자의 입력에 따라 모델이 적절한 응답을 생성합니다.

ChatGoogleGenerativeAI 클래스는 LangChain 프레임워크와 통합되어 있어, 다른 LangChain 컴포넌트와 함께 사용할 수 있습니다.

지원되는 모델 정보: https://ai.google.dev/gemini-api/docs/models/gemini?hl=ko

In [21]:
  langchain_google_genai import ChatGoogleGenerativeAI

# ChatGoogleGenerativeAI 언어 모델을 초기화합니다.
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

# 프롬프트를 전달하여 결과를 생성합니다.
answer = llm.stream("자연어처리에 대해서 간략히 설명해 줘")

# 결과를 출력합니다.
stream_response(answer)

IndentationError: unexpected indent (1727184390.py, line 1)

In [ ]:
from langchain_core.prompts import PromptTemplate

# ChatGoogleGenerativeAI 언어 모델을 초기화합니다.
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",  # 사용할 모델을 지정합니다.
)

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    "예/아니오 질문에 대답하세요. {question}는 과일입니까?"
)

# 체인을 생성합니다.
chain = prompt | model

# 결과를 출력합니다.
stream_response(chain.stream({"question": "사과"}))


## Safety Settings

Gemini 모델에는 기본 안전 설정(Satety Settings) 이 있지만, 이를 재정의할 수 있습니다.


만약 모델로부터 많은 "Safety Warnings"를 받고 있다면, 모델의 safety_settings 속성을 조정해 볼 수 있습니다.


Google의 Safety Setting Types 문서에서는 사용 가능한 카테고리와 임계값에 대한 열거형 정보를 제공합니다.


이 문서에는 콘텐츠 필터링 및 안전 설정과 관련된 다양한 카테고리와 해당 임계값이 정의되어 있어, 개발자들이 생성형 AI 모델을 활용할 때 적절한 안전 설정을 선택하고 적용하는 데 도움을 줍니다.


이를 통해 개발자들은 모델이 생성하는 콘텐츠의 안전성과 적절성을 보장하고, 사용자에게 유해하거나 부적절한 내용이 노출되는 것을 방지할 수 있습니다.

In [ ]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

llm = ChatGoogleGenerativeAI(
    # 사용할 모델을 "gemini-pro"로 지정합니다.
    model="gemini-1.5-pro-latest",
    safety_settings={
        # 위험한 콘텐츠에 대한 차단 임계값을 설정합니다.
        # 이 경우 위험한 콘텐츠를 차단하지 않도록 설정되어 있습니다. (그럼에도 기본적인 차단이 있을 수 있습니다.)
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

## Batch 단위 실행

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    # 사용할 모델을 "gemini-pro"로 지정합니다.
    model="gemini-1.5-pro-latest",
)

results = llm.batch(
    [
        "대한민국의 수도는?",
        "대한민국의 주요 관광지 5곳을 나열하세요",
    ]
)

for res in results:
    # 각 결과의 내용을 출력합니다.
    print(res.content)

## Multimodal 모델

langchain-teddynote 에서 구현한 멀티모달 모델에 gemini-1.5-pro 모델을 활용하여 이미지를 텍스트로 변환 가능합니다.

In [ ]:
from langchain_teddynote.models import MultiModal
from langchain_teddynote.messages import stream_response

# 객체 생성
gemini = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

system_prompt = (
    "당신은 시인입니다. 당신의 임무는 주어진 이미지를 가지고 시를 작성하는 것입니다."
)

user_prompt = "다음의 이미지에 대한 시를 작성해주세요."

# 멀티모달 객체 생성
multimodal_gemini = MultiModal(
    llm, system_prompt=system_prompt, user_prompt=user_prompt
)

In [ ]:
# 샘플 이미지 경로(파일의 경로, URL 등)를 지정합니다.
IMAGE_URL = "images/jeju-beach.jpg"

# 이미지 파일로 부터 질의
answer = multimodal_gemini.stream(IMAGE_URL)

# 스트리밍 방식으로 각 토큰을 출력합니다. (실시간 출력)
stream_response(answer)


# 06. 허깅페이스 엔드포인트(HuggingFace Endpoints)

Huggingface Endpoints

Hugging Face Hub은 12만 개 이상의 모델, 2만 개의 데이터셋, 5만 개의 데모 앱(Spaces)을 보유한 플랫폼으로, 모두 오픈 소스이며 공개적으로 사용 가능합니다. 이 온라인 플랫폼에서 사람들은 쉽게 협업하고 함께 머신러닝을 구축할 수 있습니다.


Hugging Face Hub은 또한 다양한 ML 애플리케이션을 구축하기 위한 다양한 엔드포인트를 제공합니다. 이 예제는 다양한 유형의 엔드포인트에 연결하는 방법을 보여줍니다.


특히, 텍스트 생성 추론은 Text Generation Inference에 의해 구동됩니다. 이는 매우 빠른 텍스트 생성 추론을 위해 맞춤 제작된 Rust, Python, gRPC 서버입니다.

사용하기 위해서는 Python의 huggingface_hub 패키지를 설치해야 합니다.

In [18]:
!pip install -U huggingface_hub

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
from huggingface_hub import login

login()


In [19]:
from langchain.prompts import PromptTemplate

template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
{question}<|end|>
<|assistant|>"""

prompt = PromptTemplate.from_template(template)

## Serverless Endpoints

Inference API 는 무료로 사용할 수 있으며 요금은 제한되어 있습니다. 프로덕션을 위한 추론 솔루션이 필요한 경우, Inference Endpoints 서비스를 확인하세요. Inference Endpoints 를 사용하면 모든 머신 러닝 모델을 전용 및 완전 관리형 인프라에 손쉽게 배포할 수 있습니다. 클라우드, 지역, 컴퓨팅 인스턴스, 자동 확장 범위 및 보안 수준을 선택하여 모델, 지연 시간, 처리량 및 규정 준수 요구 사항에 맞게 설정하세요.

다음은 Inference API 에 액세스하는 방법의 예시입니다.

repo_id 변수에 HuggingFace 모델의 repo ID(저장소 ID) 를 할당합니다.

microsoft/Phi-3-mini-4k-instruct 모델: https://huggingface.co/microsoft/Phi-3-mini-4k-instruct

In [20]:
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEndpoint

# 사용할 모델의 저장소 ID를 설정합니다.
repo_id = "microsoft/Phi-3-mini-4k-instruct"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,  # 모델 저장소 ID를 지정합니다.
    max_new_tokens=256,  # 생성할 최대 토큰 길이를 설정합니다.
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],  # 허깅페이스 토큰
)

# LLMChain을 초기화하고 프롬프트와 언어 모델을 전달합니다.
chain = prompt | llm | StrOutputParser()
# 질문을 전달하여 LLMChain을 실행하고 결과를 출력합니다.
response = chain.invoke({"question": "what is the capital of South Korea?"})
print(response)


ImportError: cannot import name 'from_env' from 'langchain_core.utils' (C:\Users\osca0\PycharmProjects\StudyLangChain\.venv\Lib\site-packages\langchain_core\utils\__init__.py)